In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Computer Algebra Systems
## Lecture 17

#### Source material
This lecture is adaption of material created by

J.R. Johansson  https://github.com/jrjohansson/scientific-python-lectures/blob/master/Lecture-5-Sympy.ipynb

and

F. Pedregosa https://github.com/scipy-lectures/scipy-lecture-notes/blob/master/packages/sympy.rst


each released under a Creative Commons Attribution License.

## Introduction

Up to now, we have been dealing with *numerical* solutions to physics problems.  In other courses on mechanics (e.g. PHYS1050, PHYS3220, PHYS3230), the approach is to seek analytical or *symbolic* solutions to problems. 

There is also a way to use computation to solve such symbolic problems.  Software that performs such tasks are called [Computer Algebra Systems](https://en.wikipedia.org/wiki/Computer_algebra_system). 

In this course, we'll will use the Python package [SymPy](http://www.sympy.org) as our computer algebra system to explore this kind of sofware and to show how it can be used to seek symbolic solutions to physics problems.

**What is SymPy?** SymPy is a Python library for symbolic mathematics.
It aims to be an alternative to systems such as Mathematica or Maple
while keeping the code as simple as possible and easily extensible.
SymPy is written entirely in Python and does not require any external
libraries.

To get started using SymPy in a Python program or notebook, import the module `sympy`:

In [ ]:
import sympy as sp

It is standard practice to abbreviate `sympy` as `sp`.

To get nice-looking $\LaTeX$ formatted output run:

In [ ]:
sp.init_printing()

## First Steps with SymPy

### Using SymPy as a calculator

SymPy defines three numerical types: `Real`, `Rational` and `Integer`.

The `Rational` class represents a rational number as a pair of two
`Integer`s: the numerator and the denominator, so `Rational(1,2)` represents
1/2, `Rational(5,2)` 5/2 and so on:

In [ ]:
a = sp.Rational(2,5)

In [ ]:
a

In [ ]:
a*5

In [ ]:
b = sp.Rational(5,12)
b

In [ ]:
a*b

SymPy uses a library for artitrary precision as numerical backend, and has predefined SymPy expressions for a number of mathematical constants, such as: `pi`, `e`, `oo` for infinity.

In [ ]:
sp.pi

In [ ]:
sp.pi**2 + sp.exp(1)

To evaluate an expression numerically we can use the `evalf` function (or `N`). It takes an argument `n` which specifies the number of significant digits.

In [ ]:
sp.pi.evalf(n=500)

In [ ]:
y = (sp.pi + sp.sqrt(5))**2

In [ ]:
y

In [ ]:
sp.N(y, 5)  # same as evalf

There is also a class representing mathematical infinity, called `oo`:

In [ ]:
sp.oo > 99999

In [ ]:
sp.oo + 1

### Exercises

1.  Calculate $\sqrt{2}$ with 100 decimals.
2.  Calculate 1/2 + 1/3 in rational arithmetic.

In [ ]:
sp.sqrt(2).evalf(100)

In [ ]:
sp.Rational(1,2) + sp.Rational(1,3)

### Working interactively with SymPy

When working interactively, it is helpful to import all of sympy into the local name space. 

In [ ]:
from sympy import *

While this is not good practice for larger programs, it definitely is more convenient:

In [ ]:
pi + sqrt(2) + N(exp(1))

We use this interactive style for the rest of this lecture.

### Symbolic variables

In SymPy we need to create symbols for the variables we want to work with. We can create a new symbol using the `Symbol` class:

In [ ]:
x = Symbol('x')

In [ ]:
(pi + x)**2

In [ ]:
# alternative way of defining symbols
a, b, c = symbols("a, b, c")

In [ ]:
type(a)

We can add assumptions to symbols when we create them:

In [ ]:
x = Symbol('x', real=True)

In [ ]:
x.is_imaginary

In [ ]:
x = Symbol('x', positive=True)

In [ ]:
x > 0

Symbols can be use to make expressions:

In [ ]:
y = (x + sqrt(pi) )**2

and can be manipulated with the standard mathematical operators:

In [ ]:
x + y + x - y

### Complex numbers

The imaginary unit is denoted `I` in Sympy. 

In [ ]:
1 + 1*I

In [ ]:
i = I

1 + i

In [ ]:
y, x = symbols("y, x")

In [ ]:
I**2

In [ ]:
(x * I + 1)**2

### Substitution

When we numerically evaluate algebraic expressions we often want to substitute a symbol with a numerical value. In SymPy we do that using the `subs` function:

In [ ]:
y = (x + pi)**2
y

In [ ]:
y.subs(x, 1.5)

In [ ]:
N(y.subs(x, 1.5))

The `subs` function can of course also be used to substitute Symbols and expressions:

In [ ]:
y.subs(x, a + pi)


We can also combine numerical evolution of expressions with NumPy arrays:

In [ ]:
x_vec = np.arange(0, 10, 0.1)

In [ ]:
y_vec = np.empty( len(x_vec) )
for i in range(len(x_vec)):
    y_vec[i] = N( y.subs(x, x_vec[i]) )

In [ ]:
fig, ax = plt.subplots()
ax.plot(x_vec, y_vec)
ax.set_title('$' + latex(y) + '$')
plt.show()

However, this kind of numerical evolution can be very slow, and there is a much more efficient way to do it: Use the function `lambdify` to "compile" a Sympy expression into a function that is much more efficient to evaluate numerically:

In [ ]:
# recall that y = (x + pi)**2 
f = lambdify([x], y, 'numpy')  # the first argument is a list of variables that
                               # f will be a function of: in this case only x -> f(x)

In [ ]:
y_vec = f(x_vec)

The speedup when using "lambdified" functions instead of direct numerical evaluation can be significant, often several orders of magnitude. Even in this simple example we get a significant speed up.

In [ ]:
%%timeit

y_vec = np.empty( len(x_vec) )
for i in range(len(x_vec)):
    y_vec[i] = N( y.subs(x, x_vec[i]) )

In [ ]:
%%timeit

y_vec = f(x_vec)

## Plotting

In addition, SymPy has its own built-in plotting functionality.

In [ ]:
plot(y)

Note that this is sp.plot() and not plt.plot() from `matplotlib`.  This can be useful for visualizing expressions.

This plotting tool can only plot univariate (one-variable) functions. So

In [ ]:
m, b = symbols('m, b')
y = m**2*x + b
y

would give an error:

In [ ]:
plot(y)

but we can supply specific values of $m$ and $b$

In [ ]:
plot(y.subs([[m, 1], [b, 5]]))

But there really is anything *special* about the symbol $x$:

In [ ]:
plot(y.subs([[x, 2], [b, 0]]))

Notice how the variable is now $m$.

## Algebraic manipulations

One of the main uses of an CAS is to perform algebraic manipulations of expressions. For example, we might want to expand a product, factor an expression, or simply an expression. The functions for doing these basic operations in SymPy are demonstrated in this section.

### Expand and factor

The first steps in an algebraic manipulation 

In [ ]:
(x+1)*(x+2)*(x+3)

In [ ]:
expand((x+1)*(x+2)*(x+3))

The `expand` function takes a number of keywords arguments which we can tell the functions what kind of expansions we want to have performed. For example, to expand trigonometric expressions, use the `trig=True` keyword argument:

In [ ]:
sin(a+b)

In [ ]:
expand(sin(a+b), trig=True)

See `help(expand)` for a detailed explanation of the various types of expansions the `expand` functions can perform.

The opposite a product expansion is of course factoring. The factor an expression in SymPy use the `factor` function: 

In [ ]:
factor(x**3 + 6 * x**2 + 11*x + 6)

### Simplify

The `simplify` tries to simplify an expression into a nice looking expression, using various techniques. More specific alternatives to the `simplify` functions also exists: `trigsimp`, `powsimp`, `logcombine`, etc. 

In [ ]:
simplify((x**3 + x**2 - x - 1)/(x**2 + 2*x + 1))

In [ ]:
# simplify uses trigonometric identities
simplify(sin(a)**2 + cos(a)**2)

### apart and together

To manipulate symbolic expressions of fractions, we can use the `apart` and `together` functions:

In [ ]:
f1 = 1/((a+1)*(a+2))

In [ ]:
f1

In [ ]:
apart(f1)

In [ ]:
f2 = 1/(a+2) + 1/(a+3)

In [ ]:
f2

In [ ]:
together(f2)

Simplify usually combines fractions.

In [ ]:
simplify(f2)

****Exercises****

> 1.  Calculate the expanded form of $(x+y)^6$.
> 2.  Simplify the trigonometric expression $\sin(x)/\cos(x)$.

In [ ]:
y = Symbol('y')
expand((x + y)**6)

In [ ]:
simplify( sin(x) / cos(x))

## Calculus

In addition to algebraic manipulations, the other main use of CAS is to do calculus, like derivatives and integrals of algebraic expressions.

### Differentiation

Differentiation is usually simple. Use the `diff` function. The first argument is the expression to take the derivative of, and the second argument is the symbol by which to take the derivative:

In [ ]:
y = (x + pi)**2
y

In [ ]:
diff(y**2, x)

For higher order derivatives we can do:

In [ ]:
diff(y**2, x, x)

In [ ]:
diff(y**2, x, 2) # same as above

To calculate the derivative of a multivariate expression, we can do:

In [ ]:
x, y, z = symbols("x,y,z")

In [ ]:
f = sin(x*y) + cos(y*z)
f

now do this: $\frac{d^3f}{dxdy^2}$

In [ ]:
diff(f, x, 1, y, 2)

## Integration

SymPy has support for indefinite and definite integration of
transcendental elementary and special functions via integrate()
facility, which uses powerful extended Risch-Norman algorithm and some
heuristics and pattern matching. You can integrate elementary functions:

In [ ]:
integrate( 6*x**5, x)

In [ ]:
f

In [ ]:
integrate(f, x)

By providing limits for the integration variable we can evaluate definite integrals:

In [ ]:
integrate(f, (x, -1, 1))

and also improper integrals

In [ ]:
Integral(exp(-x**2), (x, -oo, oo)
        ).doit()


Remember, `oo` is the SymPy notation for inifinity.

### Sums and products

We can evaluate sums and products using the functions: 'Sum'

In [ ]:
n = Symbol("n")

In [ ]:
Sum(1/n**2, (n, 1, 10))

In [ ]:
Sum(1/n**2, (n,1, 10)).evalf()

In [ ]:
Sum(1/n**2, (n, 1, oo)).doit()

Products work much the same way:

In [ ]:
Product(n, (n, 1, 10)).doit() # 10!

## Limits

Limits can be evaluated using the `limit` function. For example, 

In [ ]:
Limit(sin(x)/x, x, 0).doit()

We can use `limit` to check the result of derivation using the `diff` function:

In [ ]:
f

In [ ]:
diff(f, x)

Mathematically, a derivative is defined as a limit:

$\displaystyle \frac{\mathrm{d}f(x,y)}{\mathrm{d}x} = \lim_{h\rightarrow0} \frac{f(x+h,y)-f(x,y)}{h}$

In [ ]:
h = Symbol("h")

In [ ]:
limit((f.subs(x, x+h) - f)/h, h, 0)

Yeah! 

We can change the direction from which we approach the limiting point using the `dir` keywork argument:

In [ ]:
limit(1/x, x, 0, dir="+")

In [ ]:
limit(1/x, x, 0, dir="-")

****Exercises****

> 1.  Calculate $\lim_{x\rightarrow0} (1- \cos(x)) / x^2$.
> 2.  Calculate the derivative of $\log (x)$ with respect to x.

In [ ]:
limit((1-cos(x))/x**2, x, 0)

In [ ]:
diff(log(x), x)

## Series

Series expansion is also one of the most useful features of a CAS. In SymPy we can perform a series expansion of an expression using the `series` function:

In [ ]:
series(exp(x), x)

By default it expands the expression around $x=0$, but we can expand around any value of $x$ by explicitly include a value in the function call:

In [ ]:
series(exp(x), x, 1)

And we can explicitly define to which order the series expansion should be carried out:

In [ ]:
series(exp(x), x, 1, 10)

The series expansion includes the order of the approximation, which is very useful for keeping track of the order of validity when we do calculations with series expansions of different order:

In [ ]:
s1 = cos(x).series(x, 0, 5)
s1

In [ ]:
s2 = sin(x).series(x, 0, 2)
s2

In [ ]:
expand(s1 * s2)

If we want to get rid of the order information we can use the `removeO` method:

In [ ]:
expand(s1.removeO() * s2.removeO())

But note that this is not the correct expansion of $\cos(x)\sin(x)$ to $5$th order:

In [ ]:
(cos(x)*sin(x)).series(x, 0, 6)

## Linear algebra

### Matrices

Matrices are defined using the `Matrix` class:

In [ ]:
Matrix( [[1,0], [0,1]] )

unlike a NumPy array, you can also put Symbols in it:

In [ ]:
m11, m12, m21, m22 = symbols("m11, m12, m21, m22")
b1, b2 = symbols("b1, b2")

In [ ]:
A = Matrix([[m11, m12],[m21, m22]])
A

In [ ]:
b = Matrix([[b1], [b2]])
b

With `Matrix` class instances we can do the usual matrix algebra operations:

In [ ]:
A**2

In [ ]:
A * b

And calculate determinants and inverses, and the like:

In [ ]:
A.det()

In [ ]:
A.inv()

In [ ]:
A.eigenvals()

In [ ]:
Matrix( [[ 2, 0], [0, 2]] ).eigenvals()

Here, we are getting back a dictionary where the keys are the eigenvalues at the item values are the multiplicity of each eigenvalues.

## Solving equations

For solving equations and systems of equations we can use the `solve` function:

In [ ]:
x = Symbol('x')

In [ ]:
solve(x**2 - 1, x)

In [ ]:
solve(x**4 - x**2 - 1, x)

As you can see it takes as first argument an expression that is supposed
to be equaled to 0. It is able to solve a large part of polynomial
equations, and is also capable of solving multiple equations with
respect to multiple variables giving a tuple as second argument:

In [ ]:
solve([x + 5*y - 2, -3*x + 6*y - 15], [x, y])

In terms of other symbolic expressions:

In [ ]:
solve([x + y - a, x - y - c], [x,y])

### Exercises

> 1.  Solve the system of equations $ x + y = 2, \quad x - y = 0$.


In [ ]:
solve([ x+y -2, x-y], [x, y])

We can even combine matrices with solving equations:

In [ ]:
x1, x2 = symbols("x1, x2")
x = Matrix([x1, x2])
x

In [ ]:
solve(A*x - b, x)

## Further reading

* http://sympy.org/en/index.html - The SymPy projects web page.
* https://github.com/sympy/sympy - The source code of SymPy.
* http://live.sympy.org - Online version of SymPy for testing and demonstrations.